# Funciones de Tareas Ray

Una virtud de la API de Ray es cómo se basa en conceptos familiares que ya conoces de Python, lo que mantiene los nuevos conceptos tan intuitivos como sea posible. Si quieres encapsular algún trabajo en Python, escribe una función como ésta, que podría ser utilizada para consultar una base de datos para recuperar datos de referencia:

In [1]:
import ray
import pandas as pd
import warnings
import math
import time
import os

# Suppress noisy requests warnings.
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [2]:
ray.init()

2023-01-08 14:15:33,472	WARNING services.py:1732 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67100672 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=2.04gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-01-08 14:15:34,938	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.4
Ray version:,2.2.0
Dashboard:,http://127.0.0.1:8265


In [3]:
reference_data = [
  "Fast", "and", "Simple", "Distributed", "Computing"]

def get_ref_datum(id):
  time.sleep(id/10.0)   # Simulate an "expensive" operation
  foreign_key = 2*id
  value = reference_data[id%len(reference_data)]
  return (id, foreign_key, value)

In [4]:
start = time.time()
data = [get_ref_datum(id) for id in range(10)]
duration = time.time() - start
print(f'{duration:6.3f} secs, data = {data}')

 4.505 secs, data = [(0, 0, 'Fast'), (1, 2, 'and'), (2, 4, 'Simple'), (3, 6, 'Distributed'), (4, 8, 'Computing'), (5, 10, 'Fast'), (6, 12, 'and'), (7, 14, 'Simple'), (8, 16, 'Distributed'), (9, 18, 'Computing')]


* ¿Por qué demora 4,5 segundos? Utilizamos *slpeep* durante id/10,0 segundos. Pasamos los valores de id 0 a 9 (10 excluyente), que suman 45. Hay un poco más de tiempo tomado por otros gastos generales. Tenga en cuenta que todas las invocaciones de *get_ref_datum* son independientes entre sí, así que ¿por qué no hacerlas en paralelo? Aquí es donde entra Ray.

* Queremos convertir nuestra función síncrona, que se ejecuta en el mismo hilo que el intérprete, en una tarea que pueda ejecutarse en cualquier núcleo de CPU de nuestra máquina o en un cluster multinodo de Ray. He aquí una forma de hacerlo:

In [5]:
@ray.remote
def get_ref_datum_task_first_attempt(id):
  return get_ref_datum(id)

* El decorador *@ray.remote* marca las definiciones de tareas. Simplemente llamamos a *get_ref_datum()*. Sin embargo, esta tarea podría ejecutarse en cualquier parte de un clúster Ray, y una implementación más realista podría utilizar un gran conjunto de datos de referencia, así que vamos a utilizar nuestro almacén de objetos distribuido para mantener los reference_data y luego recuperarlos cuando sea necesario:

In [6]:
ref_data_object_ref = ray.put(reference_data)

@ray.remote
def get_ref_datum_task(id):
  time.sleep(id/10.0)   # Simulate an "expensive" operation
  foreign_key = 2*id
  ref_data = ray.get(ref_data_object_ref)
  value = ref_data[id%len(ref_data)]
  return (id, foreign_key, value)

* Los datos de referencia se colocan en el almacén de objetos distribuidos ray.put(), que devuelve una referencia de objeto de tipo ObjectRef. Los datos de referencia se recuperan del almacén de objetos con ray.get(). Hay una sobrecarga adicional al añadir esta interacción con el almacén de objetos, pero las optimizaciones internas, como la memoria compartida de copia cero, pueden eliminar gran parte de ella. Esto es especialmente importante cuando se invoca get_ref_datum_task() muchas veces concurrentemente, de modo que las instancias reales de la tarea se programan alrededor del cluster. En este caso, tendremos una sola copia de los datos de referencia en cada nodo que los necesite, en lugar de una copia por instancia de tarea.

In [7]:
start = time.time()
data_refs = [get_ref_datum_task.remote(id) for id in range(10)]
data = ray.get(data_refs)
duration = time.time() - start
print(f'{duration:6.3f} secs, data = {data}')

 1.609 secs, data = [(0, 0, 'Fast'), (1, 2, 'and'), (2, 4, 'Simple'), (3, 6, 'Distributed'), (4, 8, 'Computing'), (5, 10, 'Fast'), (6, 12, 'and'), (7, 14, 'Simple'), (8, 16, 'Distributed'), (9, 18, 'Computing')]


* Las tareas se invocan mediante un método .remote(...) añadido. En Python, sería posible que Ray modificara la función para que puedas llamar a `get_ref_datum_task(id)` directamente, pero ver el .remote(id) en el código hace obvio que Ray está en uso aquí.

* Las tareas se ejecutan de forma asíncrona, así que ¿qué se devuelve? Es un `ObjectRef` correspondiente a un futuro. Usamos `ray.get()` para recuperar la lista de objetos de los ids, que se bloquea hasta que se completan todas las tareas.

* La parte de los datos es la misma que antes, lo que indica que hemos conservado el orden de los datos resultantes, lo que podría ser importante para tu aplicación, ¡aunque hayamos ejecutado las tareas de forma asíncrona! Sin embargo, el tiempo de ejecución es ahora de sólo un segundo. ¿Por qué un segundo? Todas las tareas ahora se ejecutan en paralelo, por lo que estamos limitados por la tarea más larga, que duerme 9/10,0 segundos. Bien, ¿por qué el tiempo total no es de aproximadamente 0,9 segundos? En mi máquina de prueba, hay ocho trabajadores en ejecución. Puede ver el recuento real en su máquina en el Ray Dashboard discutido anteriormente. El valor por defecto para este número se basa en los núcleos de CPU que tiene disponibles.

* Esto significa que las ocho primeras tareas se programan inmediatamente, mientras que las dos últimas tienen que esperar a que haya trabajadores disponibles. Una tarea en espera, para id = 8, se programa muy rápidamente, porque la tarea con id = 0 termina inmediatamente. La segunda y última tarea en espera, para id = 9, espera 0,1 segundos y luego se ejecuta. Por lo tanto, su tiempo total de "reloj de pared" es 0,1 + 0,9 = 1,0 segundo.

El código que acabamos de ejecutar tiene dos inconvenientes. En primer lugar, dado que ray.get() es una llamada de bloqueo, nuestro programa principal (el "controlador" en la jerga de Ray) se queda ahí sin hacer nada mientras esperamos a que se completen las tareas. Algunas de esas tareas terminan más rápido que otras. Estaría bien procesar esos resultados mientras las tareas más lentas siguen en marcha.

En segundo lugar, en el código de producción, las llamadas que se bloquean sin establecer tiempos de espera son arriesgadas. ¿Qué pasa si una tarea remota tarda un tiempo anormalmente largo en completarse, debido a algún problema del sistema? Realmente necesitamos una forma de especificar un tiempo de espera.

Resolvamos ambos problemas con nuestro último ejemplo, que utiliza `ray.wait()`:

In [8]:
start = time.time()
data_refs = [get_ref_datum_task.remote(id) for id in range(10)]
data = []
while len(data_refs) > 0:
  finished, data_refs = ray.wait(data_refs, timeout=5.0)
  d = ray.get(finished)
  duration = time.time() - start
  print(f'  {duration:6.3f} secs, finished data = {d}')
  data.extend(d)
duration = time.time() - start
print(f'{duration:6.3f} secs, data = {data}')

   0.003 secs, finished data = [(0, 0, 'Fast')]
   0.104 secs, finished data = [(1, 2, 'and')]
   0.205 secs, finished data = [(2, 4, 'Simple')]
   0.305 secs, finished data = [(3, 6, 'Distributed')]
   0.405 secs, finished data = [(4, 8, 'Computing')]
   0.506 secs, finished data = [(5, 10, 'Fast')]
   0.607 secs, finished data = [(6, 12, 'and')]
   0.708 secs, finished data = [(7, 14, 'Simple')]
   0.808 secs, finished data = [(8, 16, 'Distributed')]
   0.910 secs, finished data = [(9, 18, 'Computing')]
 0.910 secs, data = [(0, 0, 'Fast'), (1, 2, 'and'), (2, 4, 'Simple'), (3, 6, 'Distributed'), (4, 8, 'Computing'), (5, 10, 'Fast'), (6, 12, 'and'), (7, 14, 'Simple'), (8, 16, 'Distributed'), (9, 18, 'Computing')]


* Fíjate en que el resultado de una tarea se procesa tan pronto como está lista, ¡a partir de sólo 0,002 segundos!

* El código está un poco recargado, porque también hemos añadido más sentencias print para que puedas ver lo que está pasando. El modismo central que combina ray.wait() y ray.get() es el siguiente:

In [9]:
data_refs = [get_ref_datum_task.remote(id) for id in range(10)]
data = []
while len(data_refs) > 0:
  finished, data_refs = ray.wait(data_refs, timeout=5.0)
  d = ray.get(finished)
  data.extend(d)

Seguimos haciendo todas las llamadas a .remote() a la vez, porque devuelven inmediatamente los valores ObjectRef de los futuros. Luego hacemos un bucle sobre el subconjunto de futuros que todavía se están ejecutando. ray.wait() vuelve tan pronto como una tarea termina. (Hay un argumento clave-valor opcional para especificar más de una.) Devuelve dos listas, una con las referencias a objetos terminados de las tareas terminadas y la otra con el resto de referencias de las tareas aún en ejecución. A continuación, utilizamos ray.get() para recuperar los valores de las tareas finalizadas. Ahora ray.get() no se bloqueará, porque esas tareas ya están terminadas.

Reseteamos el valor de data_refs para que sean los ids restantes, porque nunca querrás pasar referencias de tareas completadas a ray.wait(). Si lo haces, volverá inmediatamente con la primera de las referencias completadas, ¡creando un bucle infinito!

Para evitar posibles bloqueos, donde nuestra aplicación se detiene mientras algunas tareas esperan eternamente por recursos que nunca llegarán, pasamos un valor de tiempo de espera opcional de cinco segundos. Si se alcanza el tiempo de espera antes de que se complete ninguna tarea, ray.wait() devolverá una lista vacía para la primera lista.

### Dependencia de tareas

¿Qué ocurre si algunas tareas dependen de los resultados de otras tareas? En nuestro ejemplo, devolvemos una clave_externa, que utilizaremos ahora para ilustrar la elegancia con la que Ray gestiona las dependencias entre tareas. Después de recuperar un dato con una "clave externa" (en el sentido SQL del término), utilizaremos esa clave para realizar otra consulta.

Por razones que se aclararán en un momento, definiremos una segunda versión de get_ref_datum_task() que toma como argumento una tupla de tres elementos, las mismas tuplas devueltas por las llamadas a la get_ref_datum_task() original:

In [10]:
@ray.remote
def get_ref_datum_task2(tuple):
  original_id, foreign_key, value = tuple
  return tuple, get_ref_datum(foreign_key)

Esta segunda versión devuelve una "tupla de tuplas" de dos elementos: el primer elemento es la tupla de entrada, y el segundo elemento es una nueva tupla correspondiente a la clave_extranjera.

In [11]:
refs1 = [get_ref_datum_task.remote(i) for i in [2, 4, 6]]
refs2 = [get_ref_datum_task2.remote(i) for i in refs1]
for tuple in ray.get(refs2):
  print(tuple)

((2, 4, 'Simple'), (4, 8, 'Computing'))
((4, 8, 'Computing'), (8, 16, 'Distributed'))
((6, 12, 'and'), (12, 24, 'Simple'))


Para cada línea de salida, la primera tupla de tres elementos es de la invocación original a get_ref_datum_task(), y la segunda tupla de tres elementos es de la invocación posterior a get_ref_datum_task2().

La razón de la nueva definición de tarea es permitirnos simplemente pasar un ObjectRef a get_ref_datum_task2(). Ray sabe que necesitamos pasar una tupla a esta tarea, por lo que Ray llama automáticamente a ray.get() en el ObjectRef y pasa la tupla devuelta a la tarea. Ray también programa cualquier tarea get_ref_datum_task2() después de que la tarea get_ref_datum_task() correspondiente haya finalizado.

Así, Ray hace dos cosas convenientes para las tareas dependientes:

* Podemos pasar ObjectRefs a las tareas y Ray se encargará del boilerplate de extraer los resultados.

* No necesitamos esperar a que el primer conjunto de tareas se complete antes de iniciar las tareas dependientes. Ray se encarga de todo esto automáticamente.

Si este ejemplo aún no está claro, intente reescribirlo para utilizar ray.get() para recuperar primero los resultados de las llamadas a get_ref_datum_task(), bloqueando hasta que terminen. Luego pasa las tuplas resultantes explícitamente a las llamadas get_ref_datum_task2().

En otras palabras, haz el trabajo que Ray hace por ti. ¡Verás cuánta repetición no necesitas escribir! Así, incluso cuando se trabaja con tareas que tienen gráficos complejos de dependencias, escribimos código muy similar al código Python síncrono que estamos acostumbrados a escribir, sin embargo, se ejecuta a través de un clúster de forma asíncrona, con Ray gestionando las dependencias por nosotros.

### De las clases a los Ray Actors

Por muy útil que sea la abstracción de tareas, hay una deficiencia flagrante a la que se enfrentan las aplicaciones reales: ¿cómo gestionamos el estado distribuido?

El modelo de actores de Ray amplía el concepto familiar de clases orientadas a objetos para encapsular el estado y la computación.

#### ¿QUÉ SON LOS ACTORES?
El modelo de actor de concurrencia fue desarrollado en 1973 por Carl Hewitt. Los actores se comunican entre sí mediante el envío de mensajes, que se procesan en el orden en que se reciben de forma segura. De este modo, se eliminan muchos de los retos de la programación multihilo. Están diseñados para contener estados en evolución y soportar actualizaciones y accesos de forma controlada. Otras implementaciones populares de este modelo son Erlang y Akka.

Vamos a mejorar nuestra aplicación hasta ahora mediante la definición de un actor que realiza un seguimiento de los recuentos de solicitudes para cada id. Primero, empezaremos con una clase Python simple:

In [12]:
class ReferenceDataTracker:
  def __init__(self, reference_data_size):
    self.size = reference_data_size
    self.counts = [0 for _ in range(self.size)]

  def increment(self, ref):
    refmod = ref%self.size
    self.counts[refmod] += 1
    return (ref, self.counts[refmod])

Al igual que las funciones se amplían a tareas por Ray, las clases se amplían a actores. Podemos subclasificar RefDataTracker:

In [13]:
@ray.remote
class ReferenceDataTrackerActor(ReferenceDataTracker):
  def get_counts(self):
    return self.counts

RefDataTrackerActor necesita añadir un método getter para recuperar los recuentos. Ray no soporta la lectura directa de los campos del actor como en una instancia normal de Python.

La construcción del actor utiliza .remote():

In [14]:
tracker = ReferenceDataTrackerActor.remote(len(reference_data))

Por último, vamos a redefinir `get_ref_datum_task()` para invocar al actor rastreador para incrementar el recuento del id. Pasamos el rastreador como argumento.

Ten en cuenta que los métodos de actor se invocan con .remote(), igual que las tareas:

In [18]:
@ray.remote
def get_reference_datum_task(id, tracker):
  time.sleep(id/10.0)   # Simulate an "expensive" operation
  foreign_key = 2*id
  idmod = id%len(reference_data)
  value = reference_data[idmod]
  tracker.increment.remote(idmod) # Ignore returned ref
  return (id, foreign_key, value)

start = time.time()
tuple_refs = [
  get_reference_datum_task.remote(i, tracker)
    for i in range(10)]
data = ray.get(tuple_refs)
counts = ray.get(tracker.get_counts.remote())
duration = time.time() - start
print(f"""{duration:6.3f} secs:
data = {data}
counts = {counts}
""")

 0.912 secs:
data = [(0, 0, 'Fast'), (1, 2, 'and'), (2, 4, 'Simple'), (3, 6, 'Distributed'), (4, 8, 'Computing'), (5, 10, 'Fast'), (6, 12, 'and'), (7, 14, 'Simple'), (8, 16, 'Distributed'), (9, 18, 'Computing')]
counts = [6, 6, 6, 6, 6]



* Los datos son los mismos que antes. El recuento indica que cada valor se ha solicitado dos veces.

* Si ejecutas de nuevo el código anterior, los recuentos serán cuatro, porque el actor rastreador sigue ejecutándose.

Las librerías ray para ML proporcionan ejemplos más sofisticados de actores utilizados para el estado distribuido. Un ejemplo interesante es la implementación de un servidor de parámetros con un conjunto de actores. Un servidor de parámetros es una base de datos especializada para almacenar y servir parámetros de modelos gigantes. A partir de un cierto tamaño, es mejor fragmentar el conjunto de parámetros, un fragmento por actor, para mejorar el rendimiento. El estado distribuido también puede replicarse para una mayor resistencia y recuperación de errores.